In [4]:
import pandas as pd
origin =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')

In [94]:
df = origin.copy()
df.head(3)

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18


## 1. 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라

In [95]:
df['대여일자'] = pd.to_datetime(df['대여일자'])
ans = df.groupby('대여일자').size().sort_values().tail(1).index[0]
print(ans)

2021-06-04 00:00:00


## 2. 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라

In [96]:
df['day_name'] = df['대여일자'].dt.day_name()
df.value_counts('day_name')

day_name
Wednesday    110607
Friday       108877
Monday       107568
Tuesday      103704
Saturday     101299
Sunday        90918
Thursday      55977
dtype: int64

## 3. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

In [111]:
t = df.groupby(['day_name', '대여소번호']).size().to_frame('이용횟수').reset_index()
t.sort_values('이용횟수', ascending = False).drop_duplicates('day_name', keep="first")

,day_name,대여소번호,이용횟수
5191,Saturday,502,378
7611,Sunday,502,372
14820,Wednesday,502,282
348,Friday,502,277
12421,Tuesday,502,267
2769,Monday,502,242
11482,Thursday,2715,137


## 4. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

In [147]:
daily = df[df['대여구분코드'].isin(['일일권','일일권(비회원)'])].연령대코드.value_counts().sort_index()
total = df.연령대코드.value_counts().sort_index()
(daily /total).sort_values(ascending=False).index[0]


'~10대'

## 5. 연령대별 평균 이동거리를 구하여라

In [149]:
df.groupby('연령대코드').이동거리.mean()

연령대코드
20대     3211.890552
30대     3341.443859
40대     3514.857416
50대     3593.668100
60대     3538.145737
70대~    3085.039641
~10대    2634.426279
Name: 이동거리, dtype: float64

## 6. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

In [175]:
이십대 = df[df['연령대코드'] == '20대']
t = 이십대[이십대.이동거리 >= 이십대.이동거리.mean()].sort_values(['대여일자', '대여소번호'], ascending = False).head(200)
t.탄소량 = t.탄소량.astype(float)
round(t['탄소량'].mean(), 3)

1.613

## 7. 6월 7일 ~10대의 “이용건수”의 중앙값은?

In [181]:
df[(df['대여일자'] == '2021.06.07') & (df['연령대코드'] == '~10대')].이용건수.median()


1.0

## 8. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라

In [218]:
t = df[(df['day_name'].isin(['Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday'])) & (df['대여시간'].isin([6,7,8]))]
t = t.groupby(['대여시간','대여소번호']).size().to_frame('이용횟수')
t.sort_values(['대여시간','이용횟수'], ascending = False).groupby('대여시간').head(3)

이용횟수
대여시간 대여소번호      
8    2701    119
     646     115
     1152     92
7    259     104
     230      77
     726      77
6    2744     45
     1125     40
     1028     36

## 9. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

In [222]:
df[df.이동거리 >= df.이동거리.mean()].이동거리.std()

5092.139707505305

## 10. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

In [229]:
Male = df[df['성별'].isin(['M','m'])].이동거리.mean()
Female = df[df['성별'].isin(['F','f'])].이동거리.mean()

print(Male, Female)

3209.110871453049 3468.575025298235
